# L3: Multi-agent Customer Support Automation

The six key elements which help make Agents perform even better:
- Role Playing
- Focus
- Tools
- Cooperation
- Guardrails
- Memory

```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, API and LLM

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

## Role Playing, Focus and Cooperation

In [4]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

- By not setting `allow_delegation=False`, `allow_delegation` takes its default value of being `True`.
- This means the agent _can_ delegate its work to another agent which is better suited to do a particular task. 

In [5]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
)

* **Role Playing**: Both agents have been given a role, goal and backstory.
* **Focus**: Both agents have been prompted to get into the character of the roles they are playing.
* **Cooperation**: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.

## Tools, Guardrails and Memory

### Tools

- Import CrewAI tools

In [6]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

### Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

- Some ways of using CrewAI tools.

```Python
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
```

- Instantiate a document scraper tool.
- The tool will scrape a page (only 1 URL) of the CrewAI documentation.

In [7]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

##### Different Ways to Give Agents Tools

- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.

### Creating Tasks
- You are passing the Tool on the Task Level.

In [8]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

- `quality_assurance_review` is not using any Tool(s)
- Here the QA Agent will only review the work of the Support Agent

In [9]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


### Creating the Crew

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.

In [10]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=True,
  memory=True
)

### Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

#### Guardrails
- By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [11]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 052126c0-6f6d-45bf-a39b-e226844ca06d                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭────────────────────────────────────────────── 🧠 Retrieved Memory ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Historical Data:                                                                                               │
│  - Incorporate structured headings and bullet points to improve readability.                                    │
│  - Provide citations or links to specific sections of the CrewAI documentation for the user's reference.        │
│  - Include examples of code snippets for implementing memory in a crew to add hands-on guidance.                │
│  - Encourage asking follow-up questions to ensure full clarity and understanding.                               │
│  - Use a more conversational tone to increase engagement with the recipient.                                    │
│  Recent Insights:                                                                                               │
│  - I now can give a great answe...                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────── Retrieval Time: 4294.81ms ───────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Support Representative                                                                           │
│                                                                                                                 │
│  Task: DeepLearningAI just reached out with a super important ask:                                              │
│  I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you   │
│  provide guidance?                                                                                              │
│                                                                                                                 │
│  Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide     │
│  the best support possible.You must strive to provide a complete and accurate response to the customer's        │
│  inquiry.                                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

Output()

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 2c5def4e-0eea-4c78-9d57-4f13a2806fed                                                                     │
│  Agent: Support Quality Assurance Specialist                                                                    │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 052126c0-6f6d-45bf-a39b-e226844ca06d                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Hello DeepLearningAI Team,                                                                       │
│                                                                                                                 │
│  Thank you for reaching out about how to add memory to your crew and kick it off in CrewAI! I'm excited to      │
│  assist you in enhancing your experience with this platform. Below is a detailed guide to help you              │
│  successfully implement memory within your crew:                                                                │
│                                                                                                                 │
│  1. **Building the Basics:**                                                                                    │
│     To effectively add memory to your crew, it’s essential to establish a solid foundation through the          │
│  following components:                                                                                          │
│                                                                                                                 │
│     - **Agents:** Start by creating agents that incorporate tools, memory, knowledge, and structured outputs    │
│  using Pydantic. This structure will allow your agents to handle complex tasks while retaining information.     │
│  Here are some important tips:                                                                                  │
│       - Utilize templates and best practices to ensure optimal agent performance.                               │
│       - Implement memory features that enable agents to store and recall information as needed.                 │
│                                                                                                                 │
│     - **Flows:** Orchestrate steps such as start, listen, and router within your flows. This orchestration is   │
│  crucial for managing state, ensuring execution persistence, and resuming long-running workflows effectively.   │
│                                                                                                                 │
│     - **Tasks & Processes:** Define your processes in a structured manner—whether sequential, hierarchical, or  │
│  hybrid. Be sure to implement guardrails, callbacks, and human-in-the-loop triggers for better management of    │
│  the various operations your crew will handle.                                                                  │
│                                                                                                                 │
│  2. **Implementing Memory:**                                                                                    │
│     To specifically add memory to your crew, consider the following strategies:                                 │
│     - **Memory Storage Mechanism:** Implement a memory storage solution within your agents to retain context    │
│  and information across different interactions. This will allow for more meaningful conversations and task      │
│  management.                                                                                                    │
│     - **Persistent Storage Tools:** Leverage available

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

- Display the final result as Markdown.

In [12]:
from IPython.display import Markdown
Markdown(result.raw)

Hello DeepLearningAI Team,

Thank you for reaching out about how to add memory to your crew and kick it off in CrewAI! I'm excited to assist you in enhancing your experience with this platform. Below is a detailed guide to help you successfully implement memory within your crew:

1. **Building the Basics:**
   To effectively add memory to your crew, it’s essential to establish a solid foundation through the following components:

   - **Agents:** Start by creating agents that incorporate tools, memory, knowledge, and structured outputs using Pydantic. This structure will allow your agents to handle complex tasks while retaining information. Here are some important tips:
     - Utilize templates and best practices to ensure optimal agent performance.
     - Implement memory features that enable agents to store and recall information as needed.

   - **Flows:** Orchestrate steps such as start, listen, and router within your flows. This orchestration is crucial for managing state, ensuring execution persistence, and resuming long-running workflows effectively.

   - **Tasks & Processes:** Define your processes in a structured manner—whether sequential, hierarchical, or hybrid. Be sure to implement guardrails, callbacks, and human-in-the-loop triggers for better management of the various operations your crew will handle.

2. **Implementing Memory:**
   To specifically add memory to your crew, consider the following strategies:
   - **Memory Storage Mechanism:** Implement a memory storage solution within your agents to retain context and information across different interactions. This will allow for more meaningful conversations and task management.
   - **Persistent Storage Tools:** Leverage available tools and libraries that facilitate persistent storage solutions. This ensures that your crew can function effectively even after interruptions or reboots.

3. **Enterprise Journey:**
   As you explore enterprise capabilities, here are some strategies to further enhance your crew’s effectiveness:
   - **Deploy Automations:** Manage environments effectively, redeploy routines, and monitor live runs directly from the Enterprise console to improve operational efficiency.
   - **Integrate Triggers & Flows:** Establish integrations with popular platforms like Gmail, Slack, and Salesforce. This setup allows you to seamlessly pass trigger payloads into crews and flows, which streamlines operations.
   - **Team Management:** Encourage collaboration within your crew by inviting teammates, setting up role-based access control (RBAC), and effectively managing access to production automations.

I highly recommend checking out the examples and cookbooks provided in the CrewAI documentation for practical reference implementations that can guide you through the process.

If you have any more questions or need further assistance, please don't hesitate to reach out! We're dedicated to providing you with comprehensive support to ensure a successful experience with CrewAI.

Cheers,

[Senior Support Representative]  
CrewAI Support Team